# Palm Oil Futures: Part B/C/D Setup and Results

I need to first get the "all to left" dataframe. Then, I'll group by Month and take First and Last values. 

For each month, I'll calculate its gap against the previous month's end. This will then be turned into a cumulative gap by adding them up, which can be joined to the main dataframe.

In [101]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [102]:
# bring in data with path for either work or home computer, then filter to 2019 onwards and set index as Time

# palm_futures = pd.read_csv("C:/Users/jonah/Documents/A_MFRE_work_from_home/501/futures_assignment/shift_left.csv")

palm_futures = pd.read_csv("C:/Users/jizatt/Documents/Courses/FRE_501/futures_contracts/futures_assignment/shift_left.csv")

palm_futures = palm_futures[palm_futures["Time"] >= "2019-01-01"]

palm_futures.set_index("Time", inplace = True)

palm_futures.rename(columns={"cuf19": "price"}, inplace = True)

In [103]:
palm_futures.head()

price
Time              
2019-01-02  523.50
2019-01-03  519.25
2019-01-04  524.50
2019-01-07  524.75
2019-01-08  528.75

In [104]:
# Convert the index to datetime format
palm_futures.index = pd.to_datetime(palm_futures.index)

# Check if the index is ordered in ascending "Time" order
is_ordered = palm_futures.index.is_monotonic_increasing

if is_ordered:
    print("DataFrame is ordered in ascending 'Time' order.")
else:
    print("DataFrame is not ordered in ascending 'Time' order.")
    
# create Month column for Group By, Merging
palm_futures['Month'] = palm_futures.index.strftime('%Y-%m')

DataFrame is ordered in ascending 'Time' order.


In [105]:
# create dataframes of first and last trading days in each month

month_firsts  = palm_futures.groupby("Month").first()
month_lasts = palm_futures.groupby("Month").last()

# calculate profits over the month
month_profits = month_lasts["price"] - month_firsts["price"]

month_profits.head()

Month
2019-01    15.25
2019-02   -11.75
2019-03   -15.25
2019-04     8.00
2019-05   -18.50
Name: price, dtype: float64

In [106]:
# sum total monthly profits, for fun:

month_profits.sum()

383.25

Quick Peek at roll yield myth supposed profit: 

In [107]:
month_lasts["price"].tail(1)

Month
2023-08    836.0
Name: price, dtype: float64

In [108]:
month_firsts["price"].head(1)

Month
2019-01    523.5
Name: price, dtype: float64

In [109]:
836.0 - 523.5

312.5

# Calculating Synthetic Curve

The prices we observe for futures are not true representation of profits and gains from holding the asset in the long-term, due to the Roll Yield, which represents storage costs (when a positive cost) or a convenience premium for deferring commodity use (when a negative cost, or profit). 

We want to calculate the Synthetic Curve, which represents the actual path of profits, with gains or losses over time. 


`Price = Base Price + Within-Month Profit + Between-Month Gaps`

Both of these are cumulative over time. So there are actually two valid approaches to get the Synthetic Curve:

1. `Synthetic Curve` = `Price - Between-Month Gaps`

In this case, we would take each Month(i) end value and the Month(i+1) start value, and subtract the former from the latter to get their gap. We then cumulatively sum all month gaps and join these onto the main prices dataframe, and subtract the cumulative gap value from the real price curve. 

Since we've removed the `Between-Month Gaps`, we've got the part of the overall `Price` that's equal to the `Base Price` (which doesn't particularly matter for future monthly changes) and the `Within-Month Profit`, which we are actually interested in 


2. `Synthetic Curve` = `Base Price + Within-Month Profit`

There's another way we can calculate the Synthetic Curve, which is simpler conceptually, if not computationally. 

We have: 

`Price = Base Price + Within-Month Profit + Between-Month Gaps`

`Synthetic Curve = Price - Between-Month Gaps`

What if we substitute `Price` into `Synthetic Curve`?

`Synthetic Curve = Base Price + Within-Month Profit + Between-Month Gaps - Between-Month Gaps`

`Synthetic Curve = Base Price + Within-Month Profit`

So, if we take the base price at the start of the observation period (2019), and then add just the `Within-Month Profit` values, then we'd also get the Synthetic Curve.


The problem with Approach 2 is that it's easy to calculate the End-Start values for each month, and even join these to a dataframe and add them to base price, but getting the daily shifts (without the between-month shifts) is very difficult. As a result, we'll focus on removing the between-month gaps. 

### Calculating Monthly Profits

Might as well!

In [110]:
# copy month first dataframe
month_synth_prof = month_firsts.copy()

# create cumulative profit with cumulative sum of `month_profits`, the within-month profit values
month_synth_prof["cum_profit"] = month_profits.cumsum()

# calculate monthly synthetic curve
month_synth_prof["monthly_synthetic_curve"] = month_synth_prof["cum_profit"] + 523.5

month_synth_prof.head()

price  cum_profit  monthly_synthetic_curve
Month                                               
2019-01  523.50       15.25                   538.75
2019-02  562.75        3.50                   527.00
2019-03  537.50      -11.75                   511.75
2019-04  518.50       -3.75                   519.75
2019-05  507.00      -22.25                   501.25

### Approach 1: `Synthetic Curve` = `Price - Between-Month Gaps`

We will calculate the monthly gaps with `Month (i) First Value` - `Month (i-1) Last Value`. The first of these will be a `NaN` by default, and we replace it with 0. 

With these gaps, we will take a cumulative sum, and then merge them back onto the original `palm_futures` dataset at the monthly level, and subtract them from the original price series to get our `synthetic_curve`

In [111]:
month_firsts.shape

(56, 1)

In [112]:
month_lasts.shape

(56, 1)

In [113]:
month_gap = month_firsts.merge(month_lasts, on = "Month")

month_gap.rename(columns={'price_x': 'month_start', 'price_y': 'month_end'}, inplace=True)


In [114]:
month_gap.head()

month_start  month_end
Month                          
2019-01       523.50     538.75
2019-02       562.75     551.00
2019-03       537.50     522.25
2019-04       518.50     526.50
2019-05       507.00     488.50

In [115]:
month_gap["lag_end"] = month_gap["month_end"].shift(1)

month_gap.head()

month_start  month_end  lag_end
Month                                   
2019-01       523.50     538.75      NaN
2019-02       562.75     551.00   538.75
2019-03       537.50     522.25   551.00
2019-04       518.50     526.50   522.25
2019-05       507.00     488.50   526.50

In [116]:
# create month gap by subtracting row i ending value from row i+1 starting value
month_gap["gap"] = month_gap["month_start"] - month_gap["lag_end"]

# first `lag_end` value is NaN, so I'm replacing with 0
month_gap.at[month_gap.index[0], "gap"] = 0

# create cumulative month gap
month_gap["cum_gap"] = month_gap["gap"].cumsum()

# print head
month_gap.head()

month_start  month_end  lag_end    gap  cum_gap
Month                                                   
2019-01       523.50     538.75      NaN   0.00     0.00
2019-02       562.75     551.00   538.75  24.00    24.00
2019-03       537.50     522.25   551.00 -13.50    10.50
2019-04       518.50     526.50   522.25  -3.75     6.75
2019-05       507.00     488.50   526.50 -19.50   -12.75

In [117]:
# merge the month_gap dataframe onto palm_futures
palm_futures = palm_futures.reset_index().merge(month_gap, on = "Month")

palm_futures.head()

Time   price    Month  month_start  month_end  lag_end  gap  cum_gap
0 2019-01-02  523.50  2019-01        523.5     538.75      NaN  0.0      0.0
1 2019-01-03  519.25  2019-01        523.5     538.75      NaN  0.0      0.0
2 2019-01-04  524.50  2019-01        523.5     538.75      NaN  0.0      0.0
3 2019-01-07  524.75  2019-01        523.5     538.75      NaN  0.0      0.0
4 2019-01-08  528.75  2019-01        523.5     538.75      NaN  0.0      0.0

In [118]:
# calculate synthetic curve from `cuf19` minus cumulative gap
palm_futures["synthetic_curve"] = palm_futures["price"] - palm_futures["cum_gap"]

In [119]:
# Assuming you have a 'synthetic_curve' column in the DataFrame
# If not, replace 'synthetic_curve' with the actual column name
columns_to_plot = ['price', 'synthetic_curve', "cum_gap"]

# Plotting
plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
for col in columns_to_plot:
    plt.plot(palm_futures.index, palm_futures[col], label=col)

plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Line Plot of market price, synthetic_curve, cumulative month gaps')
plt.legend()
plt.grid(True)
plt.show()

# Part C Answers: Yearly Profit Plots for Palm Futures Synthetic Curve

In [120]:
palm_futures

palm_2019 = palm_futures[palm_futures["Time"] < "2020-01-01"]

palm_2020 = palm_futures[(palm_futures["Time"] >= "2020-01-01") & (palm_futures["Time"] < "2021-01-01")]

palm_2021 = palm_futures[(palm_futures["Time"] >= "2021-01-01") & (palm_futures["Time"] < "2022-01-01")]

palm_2022 = palm_futures[(palm_futures["Time"] >= "2022-01-01") & (palm_futures["Time"] < "2023-01-01")]

palm_2023 = palm_futures[(palm_futures["Time"] >= "2023-01-01")]


In [121]:
profit_2019 = palm_2019["synthetic_curve"].iloc[-1] - palm_2019["synthetic_curve"].iloc[0]
print("2019 profit is: " + str(profit_2019))

profit_2020 = palm_2020["synthetic_curve"].iloc[-1] - palm_2020["synthetic_curve"].iloc[0]
print("2020 profit is: " + str(profit_2020))

profit_2021 = palm_2021["synthetic_curve"].iloc[-1] - palm_2021["synthetic_curve"].iloc[0]
print("2021 profit is: " + str(profit_2021))

profit_2022 = palm_2022["synthetic_curve"].iloc[-1] - palm_2022["synthetic_curve"].iloc[0]
print("2022 profit is: " + str(profit_2022))

profit_2023 = palm_2023["synthetic_curve"].iloc[-1] - palm_2023["synthetic_curve"].iloc[0]
print("2023 profit is: " + str(profit_2023))

profit_overall = palm_futures["synthetic_curve"].iloc[-1] - palm_futures["synthetic_curve"].iloc[0]
print("Overall profit is: " + str(profit_overall))

2019 profit is: 121.0
2020 profit is: 171.0
2021 profit is: 381.5
2022 profit is: -251.25
2023 profit is: -39.0
Overall profit is: 383.25


In [122]:

# Set seaborn style
sns.set(style="whitegrid")

# Define the profit values and corresponding years
profit_values = [profit_2019, profit_2020, profit_2021, profit_2022, profit_2023, profit_overall]
years = ['2019', '2020', '2021', '2022', '2023', 'Overall']

# Define custom colors
colors = ['#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', '#d62728']

# Create a bar plot
plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
bars = plt.bar(years, profit_values, color=colors)

# Add value labels above each bar
for bar, value in zip(bars, profit_values):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 5, f'{value:.2f}', ha='center', va='bottom', fontweight='bold')

plt.xlabel('Year')
plt.ylabel('Profit')
plt.title('Profit Over Years', fontsize=16, fontweight='bold')
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.savefig("professional_plot.png", dpi=300)  # Save the plot to a file at high resolution
plt.show()

# Export Cumulative Gap


In [123]:
palm_futures

Time   price    Month  month_start  month_end  lag_end   gap  \
0    2019-01-02  523.50  2019-01        523.5     538.75      NaN  0.00   
1    2019-01-03  519.25  2019-01        523.5     538.75      NaN  0.00   
2    2019-01-04  524.50  2019-01        523.5     538.75      NaN  0.00   
3    2019-01-07  524.75  2019-01        523.5     538.75      NaN  0.00   
4    2019-01-08  528.75  2019-01        523.5     538.75      NaN  0.00   
...         ...     ...      ...          ...        ...      ...   ...   
1162 2023-08-15  825.25  2023-08        859.5     836.00   860.25 -0.75   
1163 2023-08-16  828.50  2023-08        859.5     836.00   860.25 -0.75   
1164 2023-08-17  835.75  2023-08        859.5     836.00   860.25 -0.75   
1165 2023-08-18  831.50  2023-08        859.5     836.00   860.25 -0.75   
1166 2023-08-21  836.00  2023-08        859.5     836.00   860.25 -0.75   

      cum_gap  synthetic_curve  
0        0.00           523.50  
1        0.00           519.25  
2        0.00           524.50  
3        0.00           524.75  
4        0.00           528.75  
...       ...              ...  
1162   -70.75           896.00  
1163   -70.75           899.25  
1164   -70.75           906.50  
1165   -70.75           902.25  
1166   -70.75           906.75  

[1167 rows x 9 columns]

In [124]:
cumulative_gap = palm_futures[["Time", "Month", "cum_gap"]]

In [125]:
cumulative_gap

Time    Month  cum_gap
0    2019-01-02  2019-01     0.00
1    2019-01-03  2019-01     0.00
2    2019-01-04  2019-01     0.00
3    2019-01-07  2019-01     0.00
4    2019-01-08  2019-01     0.00
...         ...      ...      ...
1162 2023-08-15  2023-08   -70.75
1163 2023-08-16  2023-08   -70.75
1164 2023-08-17  2023-08   -70.75
1165 2023-08-18  2023-08   -70.75
1166 2023-08-21  2023-08   -70.75

[1167 rows x 3 columns]

In [127]:
#cumulative_gap.to_csv("cumulative_gap.csv")


# Part C Join and Merge

pd.merge(palm_futures, cumulative_gap, on = "Time")

In [128]:
cumulative_gap.set_index("Time").join(palm_futures.set_index("Time"), on = "Time")

ValueError: columns overlap but no suffix specified: Index(['Month', 'cum_gap'], dtype='object')

# Part B Answer

In [130]:
# load url and adjust as path
url = "https://drive.google.com/file/d/18SdzXz7w9a4yEIEsfrvF2dXTmLDJ-pSF/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split('/')[-2]

# import then open first rows
palm_futures = pd.read_csv(path)

In [131]:
palm_futures

trade_day   cuf19   cug19   cuh19   cuj19   cuk19   cum19   cun19   cuq19  \
0           1  523.50  562.75  537.50  518.50  507.00  484.75  474.00  498.25   
1           2  519.25  563.75  528.00  526.75  491.25  491.75  475.25  496.75   
2           3  524.50  563.75  530.25  531.75  485.50  491.75  472.75  501.25   
3           4  524.75  563.75  528.25  538.25  479.25  491.75  474.00  501.25   
4           5  528.75  567.00  524.00  542.00  490.75  487.75  470.25  501.25   
5           6  529.00  562.75  521.50  536.00  490.25  480.75  469.75  508.00   
6           7  527.00  558.25  520.75  538.25  484.25  483.00  469.00  515.25   
7           8  528.50  555.00  521.00  531.00  480.00  476.25  471.00  515.25   
8           9  523.25  555.50  516.50  527.25  480.00  482.50  472.25  519.75   
9          10  524.50  559.75  513.00  528.00  484.25  482.50  474.50  521.25   
10         11  527.25  555.00  513.00  526.00  485.50  485.25  477.25  514.75   
11         12  530.00  556.00  517.75  529.50  494.00  483.50  477.75  517.50   
12         13  532.75  555.50  521.00  531.50  493.75  486.25  477.50  514.25   
13         14  535.50  557.25  524.25  529.00  493.75  485.00  476.50  515.75   
14         15  536.75  556.25  525.50  528.75  489.25  485.00  477.25  519.00   
15         16  537.75  554.00  525.00  528.75  489.25  483.75  478.75  522.75   
16         17  537.50  552.75  523.25  528.75  485.50  483.00  480.25  522.75   
17         18  539.75  551.25  523.00  528.25  485.25  482.00  481.75  522.50   
18         19  538.50  551.00  523.00  526.75  488.00  482.00  482.00  521.25   
19         20  538.75     NaN  522.50  526.75  489.25  482.00  481.00  521.75   
20         21  538.75     NaN  522.25  526.50  488.75     NaN  481.00  520.75   
21         22     NaN     NaN     NaN     NaN  488.50     NaN  481.00  521.00   
22         23     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

     cuu19  ...   cux22   cuz22   cuf23   cug23   cuh23   cuj23   cuk23  \
0   516.25  ...  897.00  926.50  964.50  897.25  929.50  868.50  747.25   
1   519.00  ...  928.75  901.75  945.25  887.50  955.50  888.75  762.25   
2   520.25  ...  917.25  911.75  931.00  902.25  967.75  870.50  762.50   
3   525.25  ...  923.50  934.25  925.25  902.25  955.75  859.75  762.50   
4   525.25  ...  935.25  908.50  941.75  913.50  940.00  858.25  799.50   
5   529.75  ...  923.25  902.75  921.00  925.50  929.25  863.75  830.00   
6   526.50  ...  898.75  913.25  908.25  919.75  931.25  849.00  832.00   
7   524.25  ...  898.25  870.00  909.50  909.25  913.75  843.75  816.75   
8   525.25  ...  922.75  888.50  903.50  904.25  908.50  842.00  798.75   
9   525.25  ...  907.25  899.25  898.75  909.75  901.75  846.75  802.25   
10  536.75  ...  896.50  890.75  905.25  904.75  906.75  859.75  794.50   
11  533.50  ...  898.50  891.50  908.25  915.75  899.50  856.25  787.25   
12  531.25  ...  880.00  893.00  910.25  919.75  899.75  854.50  782.75   
13  530.00  ...  880.00  887.00  910.25  920.75  887.50  854.50  778.00   
14  527.75  ...  878.50  893.50  910.25  920.75  887.50  854.50  784.50   
15  524.25  ...  889.50  889.75  902.75  925.50  878.25  850.75  781.00   
16  524.25  ...  896.00  886.25  905.00  924.25  874.00  847.50  776.25   
17  525.00  ...  901.75  898.25  910.25  924.00  871.00  844.75  776.75   
18  524.50  ...  901.75  897.50  911.00  923.00  874.00  843.25  780.00   
19  524.25  ...  901.25  897.75  909.25     NaN  879.50     NaN  782.25   
20     NaN  ...  901.50  898.75     NaN     NaN  880.00     NaN  779.25   
21     NaN  ...     NaN     NaN     NaN     NaN  881.25     NaN  777.25   
22     NaN  ...     NaN     NaN     NaN     NaN  881.00     NaN     NaN   

     cum23   cun23   cuq23  
0   711.75  852.75  859.50  
1   734.75  833.00  865.00  
2   734.75  839.50  843.25  
3   721.25  826.25  851.75  
4   721.00  840.50  836.00  
5   710.75  835.25  818.50  
6   727.50  842.00  830.50  
7   723.50  839.00  824.50

In [132]:
# Calculate the difference between the first non-NaN and last non-NaN values in each column
def calculate_difference(column):
    first_non_nan = column.dropna().iloc[0]
    last_non_nan = column.dropna().iloc[-1]
    return last_non_nan - first_non_nan

# Apply the custom function to each column in the DataFrame
df_diff = palm_futures.drop("trade_day", axis=1).apply(calculate_difference)

# Create a dictionary to store the summed values for each group
summed_values = {}

# Iterate through the columns and group by the last two characters in the column name
for column in df_diff.index:
    group_key = column[-2:]  # Extract the last two characters
    if group_key not in summed_values:
        summed_values[group_key] = df_diff[column]
    else:
        summed_values[group_key] += df_diff[column]

# Create a new DataFrame from the summed_values dictionary
result_df = pd.DataFrame(list(summed_values.items()), columns=['Group', 'Sum'])

# Print the result
print(result_df)

  Group     Sum
0    19  121.00
1    20  171.00
2    21  381.50
3    22 -251.25
4    23  -39.00


# Part D

### 1. Simple approach

`year_status` is 1 when I eyeball the year as in Contango, 0 when Ambiguous, and -1 when in Backwardation.

In [133]:
year_profits = pd.Series([121.0, 171.0, 381.50, -251.25, -39.0])

year_status = pd.Series([1, 0, -1, -1, -1])

year_profits.corr(year_status)

0.21587252101299106

So, Contango is positively correlated with observed profits. Since the Yield Curve Myth is that you suffer instant losses from contract rollovers, this acts as weak evidence against it (due to the aggregation and small sample size).

Notably, this is highly sensitive to classifying 2022 as in Backwardation or Ambiguous, as the result changes substantially:

In [134]:
year_status_2 = pd.Series([1, 0, -1, 0, -1])

year_profits.corr(year_status_2)

-0.1826173780954296

### 2. Complicated Approach

In [135]:
palm_futures

trade_day   cuf19   cug19   cuh19   cuj19   cuk19   cum19   cun19   cuq19  \
0           1  523.50  562.75  537.50  518.50  507.00  484.75  474.00  498.25   
1           2  519.25  563.75  528.00  526.75  491.25  491.75  475.25  496.75   
2           3  524.50  563.75  530.25  531.75  485.50  491.75  472.75  501.25   
3           4  524.75  563.75  528.25  538.25  479.25  491.75  474.00  501.25   
4           5  528.75  567.00  524.00  542.00  490.75  487.75  470.25  501.25   
5           6  529.00  562.75  521.50  536.00  490.25  480.75  469.75  508.00   
6           7  527.00  558.25  520.75  538.25  484.25  483.00  469.00  515.25   
7           8  528.50  555.00  521.00  531.00  480.00  476.25  471.00  515.25   
8           9  523.25  555.50  516.50  527.25  480.00  482.50  472.25  519.75   
9          10  524.50  559.75  513.00  528.00  484.25  482.50  474.50  521.25   
10         11  527.25  555.00  513.00  526.00  485.50  485.25  477.25  514.75   
11         12  530.00  556.00  517.75  529.50  494.00  483.50  477.75  517.50   
12         13  532.75  555.50  521.00  531.50  493.75  486.25  477.50  514.25   
13         14  535.50  557.25  524.25  529.00  493.75  485.00  476.50  515.75   
14         15  536.75  556.25  525.50  528.75  489.25  485.00  477.25  519.00   
15         16  537.75  554.00  525.00  528.75  489.25  483.75  478.75  522.75   
16         17  537.50  552.75  523.25  528.75  485.50  483.00  480.25  522.75   
17         18  539.75  551.25  523.00  528.25  485.25  482.00  481.75  522.50   
18         19  538.50  551.00  523.00  526.75  488.00  482.00  482.00  521.25   
19         20  538.75     NaN  522.50  526.75  489.25  482.00  481.00  521.75   
20         21  538.75     NaN  522.25  526.50  488.75     NaN  481.00  520.75   
21         22     NaN     NaN     NaN     NaN  488.50     NaN  481.00  521.00   
22         23     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

     cuu19  ...   cux22   cuz22   cuf23   cug23   cuh23   cuj23   cuk23  \
0   516.25  ...  897.00  926.50  964.50  897.25  929.50  868.50  747.25   
1   519.00  ...  928.75  901.75  945.25  887.50  955.50  888.75  762.25   
2   520.25  ...  917.25  911.75  931.00  902.25  967.75  870.50  762.50   
3   525.25  ...  923.50  934.25  925.25  902.25  955.75  859.75  762.50   
4   525.25  ...  935.25  908.50  941.75  913.50  940.00  858.25  799.50   
5   529.75  ...  923.25  902.75  921.00  925.50  929.25  863.75  830.00   
6   526.50  ...  898.75  913.25  908.25  919.75  931.25  849.00  832.00   
7   524.25  ...  898.25  870.00  909.50  909.25  913.75  843.75  816.75   
8   525.25  ...  922.75  888.50  903.50  904.25  908.50  842.00  798.75   
9   525.25  ...  907.25  899.25  898.75  909.75  901.75  846.75  802.25   
10  536.75  ...  896.50  890.75  905.25  904.75  906.75  859.75  794.50   
11  533.50  ...  898.50  891.50  908.25  915.75  899.50  856.25  787.25   
12  531.25  ...  880.00  893.00  910.25  919.75  899.75  854.50  782.75   
13  530.00  ...  880.00  887.00  910.25  920.75  887.50  854.50  778.00   
14  527.75  ...  878.50  893.50  910.25  920.75  887.50  854.50  784.50   
15  524.25  ...  889.50  889.75  902.75  925.50  878.25  850.75  781.00   
16  524.25  ...  896.00  886.25  905.00  924.25  874.00  847.50  776.25   
17  525.00  ...  901.75  898.25  910.25  924.00  871.00  844.75  776.75   
18  524.50  ...  901.75  897.50  911.00  923.00  874.00  843.25  780.00   
19  524.25  ...  901.25  897.75  909.25     NaN  879.50     NaN  782.25   
20     NaN  ...  901.50  898.75     NaN     NaN  880.00     NaN  779.25   
21     NaN  ...     NaN     NaN     NaN     NaN  881.25     NaN  777.25   
22     NaN  ...     NaN     NaN     NaN     NaN  881.00     NaN     NaN   

     cum23   cun23   cuq23  
0   711.75  852.75  859.50  
1   734.75  833.00  865.00  
2   734.75  839.50  843.25  
3   721.25  826.25  851.75  
4   721.00  840.50  836.00  
5   710.75  835.25  818.50  
6   727.50  842.00  830.50  
7   723.50  839.00  824.50

In [136]:
# create month_profit variable

month_gap["month_profit"] = month_gap["month_end"] - month_gap["month_start"]

month_gap.head()

month_start  month_end  lag_end    gap  cum_gap  month_profit
Month                                                                 
2019-01       523.50     538.75      NaN   0.00     0.00         15.25
2019-02       562.75     551.00   538.75  24.00    24.00        -11.75
2019-03       537.50     522.25   551.00 -13.50    10.50        -15.25
2019-04       518.50     526.50   522.25  -3.75     6.75          8.00
2019-05       507.00     488.50   526.50 -19.50   -12.75        -18.50

In [137]:
month_gap["month_profit"].corr(month_gap["gap"])

-0.12488701740050849

A positive (negative) gap implies the market is in Contango (Backwardation).

The Roll Yield Myth asserts that the roll yield is negative in Contango market because the offset price of the expiring long contract is less than the price of the long position in the new contract.

Conversely, it asserts that the roll yield is positive in Backwardation market.

The correlation of -0.125 lends weak support to the RYM, which sugggests negative correlation of market direction and profits. 

However, in class better testing methods will show that the negative outcome has nothing to do with the size of the gap when the roll takes place, which is why the RYM is a Myth.

In [138]:
month_gap["month_profit"].plot()
month_gap["month_start"].plot()

<AxesSubplot: xlabel='Month'>

In [139]:
month_gap.head()

month_start  month_end  lag_end    gap  cum_gap  month_profit
Month                                                                 
2019-01       523.50     538.75      NaN   0.00     0.00         15.25
2019-02       562.75     551.00   538.75  24.00    24.00        -11.75
2019-03       537.50     522.25   551.00 -13.50    10.50        -15.25
2019-04       518.50     526.50   522.25  -3.75     6.75          8.00
2019-05       507.00     488.50   526.50 -19.50   -12.75        -18.50

In [140]:
month_start_end = month_gap[["month_start", "month_end"]]

gap = month_gap["gap"]

month_start_end.head()

month_start  month_end
Month                          
2019-01       523.50     538.75
2019-02       562.75     551.00
2019-03       537.50     522.25
2019-04       518.50     526.50
2019-05       507.00     488.50

In [141]:
gap.head()

Month
2019-01     0.00
2019-02    24.00
2019-03   -13.50
2019-04    -3.75
2019-05   -19.50
Name: gap, dtype: float64

In [142]:
month_start_end.to_csv("month_start_end.csv")

gap.to_csv("gap.csv")